# What's In A Name? Investigating Bias And Identity In Text-To-Image Models

## Introduction

This work will investigate the relationships between names and biases in text to image models. some narrower topics of interest include:

* constructing measures for characterizing the degree to which a model has learned the relationship between a name and specific entity
* measuring biases associated with names that are not tightly bound to identities
* characterizing how biases of these kinds can be composed into new identities
* characterizing the "stickiness" of specific attribute biases to identities/names
* investigating the potential for leveraging these measure to monitor training progress for personalization finetuning and general purpose pre-training


* biases in names
* known identities ("strong entities")
* overfitted identities (memorized images)
  * mona lisa?

* Just investigating biases, not making normative statements

* associated hypotheses
* out of scope hypotheses (future work)

## Methodology [wip]

* choosing names
* generating images

## Experiments

* calibrating minimum # images
* investigating which embedding representation to use
* experimenting with summary statistics

## Insights

* presumptive phases of bias formation
* correlation between age bias and popularity of names by birth year
* strength of ethnic bias reflects undersampled data class
  * strong biases hint at low data cardinality for the given class


## Discussion


## Future Work


## References

* CLIP
* Stable Diffusion
* Stable Biases
* Celeb Basis

## Appendix

* Calibration plots
* DINO-vits stuff (assuming DINO-vitg shows in main report)
